In [4]:
import pandas as pd
import numpy as np

In [ ]:
data = 'accepted_2007_to_2018Q4.csv'

In [ ]:
data = pd.read_csv(data, low_memory=False)

In [ ]:
data.head()

In [ ]:
# Set your NA threshold (e.g., remove columns with more than 50% missing)
threshold = 0.3
na_fraction = upsampled.isnull().mean()

# Drop columns above the threshold
upsampled_cleaned = upsampled.drop(columns=na_fraction[na_fraction > threshold].index)

# Preview cleaned DataFrame
print(upsampled_cleaned.shape)
print(upsampled_cleaned.head())


In [ ]:

print(f"After dropna: {upsampled_cleaned.shape}")

In [ ]:
# Drop them from your DataFrame
columns_to_drop = ['emp_title', 'sub_grade']
upsampled_cleaned = upsampled_cleaned.drop(columns=columns_to_drop)

# Preview the cleaned DataFrame
print(upsampled_cleaned.head())


In [ ]:
# ✅ Convert 'Y'/'N' columns to 1/0
yn_cols = ['debt_settlement_flag', 'hardship_flag']  # Add more columns if needed
for col in yn_cols:
    upsampled_cleaned[col] = upsampled_cleaned[col].map({'Y': 1, 'N': 0})

# 💳 Convert 'disbursement_method' to binary (Cash = 1, others = 0)
upsampled_cleaned['disbursement_method'] = (upsampled_cleaned['disbursement_method'] == 'Cash').astype(int)
upsampled_cleaned['home_ownership'] = (upsampled_cleaned['home_ownership'] == 'MORTGAGE').astype(int)

# 📅 Convert 'emp_length' like '10+ years', '< 1 year', 'n/a' to numeric
def clean_emp_length(val):
    if pd.isna(val): return np.nan
    val = str(val).lower().strip()
    if '<' in val: return 0
    if '10+' in val: return 10
    if 'n/a' in val: return np.nan
    digits = ''.join(filter(str.isdigit, val))
    return int(digits) if digits else np.nan

upsampled_cleaned['emp_length'] = upsampled_cleaned['emp_length'].apply(clean_emp_length)
# Remove 'months' and convert to integer
upsampled_cleaned['term'] = upsampled_cleaned['term'].str.extract('(\d+)').astype(int)




In [ ]:
# 🅰️ Map loan grade (A-G) to integers
grade_map = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7}
upsampled_cleaned['grade_num'] = upsampled_cleaned['grade'].map(grade_map)
upsampled_cleaned = upsampled_cleaned.drop(columns='grade')

In [ ]:
# Drop the last two rows of the DataFrame
upsampled_cleaned = upsampled_cleaned.iloc[:-2]

In [ ]:
# Extract numeric part and convert safely

#upsampled_cleaned['term'] = upsampled_cleaned['term'].str.extract('(\d+)')
upsampled_cleaned['term'] = upsampled_cleaned['term'].astype('Int64')  # Capital I!

In [ ]:
print(upsampled_cleaned.head())

In [ ]:
summary = pd.DataFrame({
    'dtype': upsampled_cleaned.dtypes,
    'missing_frac': upsampled_cleaned.isnull().mean()
})
print(summary.sort_values('missing_frac', ascending=False))


In [ ]:
print(upsampled_cleaned['loan_status'].head())

In [ ]:
upsampled_cleaned_targeted = upsampled_cleaned
upsampled_cleaned_targeted['loan_status'] = upsampled_cleaned['loan_status'].str.strip().str.lower()
upsampled_cleaned_targeted.head()

In [ ]:
# Define default statuses
default_statuses = [
    'charged off',
    'default',
    'does not meet the credit policy. status:charged off'
]

non_default_statuses = [
    'fully paid',
    'does not meet the credit policy. status:fully paid'
]
# Filter to only default/non-default loans
upsampled_cleaned_targeted = upsampled_cleaned_targeted[upsampled_cleaned['loan_status'].isin(default_statuses + non_default_statuses)].copy()

# Create binary target
upsampled_cleaned_targeted['is_default'] = upsampled_cleaned_targeted['loan_status'].isin(default_statuses).astype(int)

# Check the distribution
print(upsampled_cleaned_targeted['is_default'].value_counts(normalize=True))


In [ ]:
# Keep only numeric (int, float, bool) columns
numeric_df = upsampled_cleaned_targeted.select_dtypes(include=['number', 'bool'])

# View remaining columns



In [ ]:
print(numeric_df)

In [ ]:
# Save cleaned dataset to CSV
numeric_df.to_csv('Kaggle_full_cleaned_dataset.csv', index=False)

# Optional: confirm save
print("Saved cleaned dataset with shape:", upsampled_cleaned.shape)


In [ ]:
# Selecting only the most common variables

In [18]:
data = pd.read_csv('Kaggle_full_cleaned_dataset.csv')

In [20]:
# Define Tier 1 features
tier_1_features = [
    'int_rate'
    'loan_amnt',
    'funded_amnt',
    'term',
    'installment',
    'annual_inc',
    'dti',
    'emp_length',
    'home_ownership',
    'fico_range_low',
    'fico_range_high',
    'inq_last_6mths',
    'open_acc',
    'pub_rec',
    'revol_bal',
    'revol_util',
    'total_acc',
    'is_default'
]

# Create new DataFrame with only Tier 1 features (if present in your data)
tier_1_df = data[[col for col in tier_1_features if col in data.columns]]

# Optional: save to CSV
tier_1_df.to_csv("tier_1_features_only.csv", index=False)
